In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Change the path to local directory where the file will be saved that needs to be read.
path = 'C:/Users/91880/Desktop/karan/Dissertation'

In [3]:
#Import necessary libraries and sklearn modules to run the process.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [4]:
# Read csv file.
ehw = pd.read_csv(path + '/ehw.csv', encoding= 'unicode_escape')

In [5]:
#This is how file looks like.
ehw.head()

,LCLid,time,energy_sum,stdorToU,Acorn,Acorn_grouped,temperatureMax,windBearing,icon,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase
0,MAC002542,17-Jan-13,5.543,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
1,MAC002543,17-Jan-13,5.270,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
2,MAC002545,17-Jan-13,2.015,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
3,MAC002546,17-Jan-13,7.329,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
4,MAC002548,17-Jan-13,4.479,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2


In [6]:
#Picking up random 80,000 rows for faster processing as the dataset is quite large.
ehw1 = ehw.sample(n = 80000)

In [7]:
#The summary of whole dataset's statistical properties. 
#This helps to verify the count and look for any missing values if present, mean,max etc.
ehw1.describe()

,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,9.627996,14.659034,194.825288,7.277976,0.480186,3.465287,1014.247530,11.297605,0.776260,2.863375,8.237336,0.504949
std,9.372916,6.653442,92.190170,5.096565,0.191673,1.621608,9.880922,2.370434,0.099182,1.912473,5.162989,0.288403
min,0.000000,-0.060000,0.000000,-7.840000,0.000000,0.400000,979.520000,1.480000,0.430000,0.000000,-5.640000,0.000000
25%,4.157000,9.780000,113.000000,3.680000,0.350000,2.310000,1008.580000,10.460000,0.710000,1.000000,4.040000,0.260000
50%,6.999000,14.560000,218.000000,7.790000,0.470000,3.360000,1014.620000,12.050000,0.780000,2.000000,8.480000,0.510000
75%,11.757250,19.640000,257.000000,11.200000,0.600000,4.410000,1020.900000,12.860000,0.860000,5.000000,12.480000,0.760000
max,306.435000,32.400000,359.000000,17.770000,1.000000,9.490000,1040.920000,15.340000,0.980000,7.000000,20.540000,0.990000


In [8]:
#Displaying the correlation between different variables.
# This helps to understand the relationship of all features with energy.Closer to 1 means the stronger the correlation.
# A minus sign indicates negative correlation between the two variables, which suggests that if one increases, the other lowers and vice versa.
ehw1.corr()

,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase
energy_sum,1.000000,-0.178269,-0.015983,-0.159644,0.064874,0.025768,-0.023009,-0.067677,0.080777,-0.138979,-0.171533,-0.001970
temperatureMax,-0.178269,1.000000,0.090199,0.866362,-0.398682,-0.168135,0.155930,0.313132,-0.428154,0.685801,0.902902,0.021555
windBearing,-0.015983,0.090199,1.000000,0.118952,-0.080200,0.021026,0.003923,0.245798,0.017906,-0.046238,0.092019,0.047996
dewPoint,-0.159644,0.866362,0.118952,1.000000,-0.102313,-0.115930,-0.002631,0.068075,0.034155,0.467750,0.940606,0.008324
cloudCover,0.064874,-0.398682,-0.080200,-0.102313,1.000000,0.190045,-0.109185,-0.371582,0.478469,-0.289279,-0.134635,-0.083599
windSpeed,0.025768,-0.168135,0.021026,-0.115930,0.190045,1.000000,-0.278023,0.232834,-0.027714,-0.098292,-0.050442,0.028033
pressure,-0.023009,0.155930,0.003923,-0.002631,-0.109185,-0.278023,1.000000,0.076001,-0.312104,0.128948,0.052013,0.053491
visibility,-0.067677,0.313132,0.245798,0.068075,-0.371582,0.232834,0.076001,1.000000,-0.612256,0.312176,0.215721,0.086547
humidity,0.080777,-0.428154,0.017906,0.034155,0.478469,-0.027714,-0.312104,-0.612256,1.000000,-0.573229,-0.224790,-0.028106
uvIndex,-0.138979,0.685801,-0.046238,0.467750,-0.289279,-0.098292,0.128948,0.312176,-0.573229,1.000000,0.591424,-0.002326


In [9]:
#Dtypes function shows the data types of each variable.
ehw1.dtypes

LCLid              object
time               object
energy_sum        float64
stdorToU           object
Acorn              object
Acorn_grouped      object
temperatureMax    float64
windBearing         int64
icon               object
dewPoint          float64
cloudCover        float64
windSpeed         float64
pressure          float64
visibility        float64
humidity          float64
uvIndex             int64
temperatureMin    float64
moonPhase         float64
dtype: object

In [10]:
#Changing data type from object to int.
ehw1.loc[:,'time']=  pd.to_datetime(ehw1['time']).dt.date
#Using label encoder to handel categorical values.
labelencoder = LabelEncoder()
ehw1['Acorn' +'_']= labelencoder.fit_transform(ehw1['Acorn'])
ehw1['Acorn_grouped' +'_']= labelencoder.fit_transform(ehw1['Acorn_grouped'])
ehw1['time' +'_']= labelencoder.fit_transform(ehw1['time'])
ehw1['stdorToU' +'_']= labelencoder.fit_transform(ehw1['stdorToU'])
ehw1['icon' +'_']= labelencoder.fit_transform(ehw1['icon'])
ehw1['LCLid' +'_']= labelencoder.fit_transform(ehw1['LCLid'])
features = ['Acorn', 'Acorn_grouped','time', 'stdorToU', 'icon','LCLid']  
ehw1 = ehw1.drop(features, axis = 1)

In [11]:
ehw1.dtypes

energy_sum        float64
temperatureMax    float64
windBearing         int64
dewPoint          float64
cloudCover        float64
windSpeed         float64
pressure          float64
visibility        float64
humidity          float64
uvIndex             int64
temperatureMin    float64
moonPhase         float64
Acorn_              int32
Acorn_grouped_      int32
time_               int32
stdorToU_           int32
icon_               int32
LCLid_              int32
dtype: object

In [12]:
# Getting X and Y from ehw1 with all features.
X1 = ehw1.iloc[:,1:18] 
Y1 = ehw1.iloc[:,0] 

In [13]:
#Splitting dataset in training and testing
X_train,X_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.2,random_state=2005)

In [14]:
#Calculating Mean square error using Linear Regression without feature selection.
lin1 = LinearRegression()
lin1.fit(X_train,y_train)
lin1_y_pred = lin1.predict(X_test)
L1 = mean_squared_error(y_test,lin1_y_pred)
L1

74.04266755284492

In [15]:
#Calculating Mean square error using KNN Regression without feature selection.
knn1 = KNeighborsRegressor(n_neighbors=20)
knn1.fit(X_train, y_train)
knn1_y_pred = knn1.predict(X_test)
# Squared Mean Error score and Root Mean Squared Error:
K1 = mean_squared_error(y_test, knn1_y_pred)
K1

78.93085295687098

In [16]:
#Calculating Mean square error using Ridge Regression without feature selection.
ridg1 = Ridge(alpha=1.0)
ridg1.fit(X_train,y_train)
ridg1_y_pred = ridg1.predict(X_test)
R1 = mean_squared_error(y_test,ridg1_y_pred)
R1

74.04314633159025

## Using Feature Selection

In [17]:
#Duplicating dataset for selecting feautres 
ehw2=ehw1

In [18]:
#Getting X and Y from ehw2
X_FS= ehw2.iloc[:,1:18]  
Y_FS= ehw2.iloc[:,0] 
Y_FS=Y_FS.astype('int')

In [19]:
#Removing values that are negative because feature selection did not work with negative values.
X_FS = X_FS[(X_FS > 0).all(1)]

In [20]:
# Finding Index's of values that are non-negative in X_FS and getting their respective labels from Y_FS.
a = X_FS.index
b = []
for i in a:
    b+=[i]
Y_FS = Y_FS.loc[b]

In [21]:
X_FS

,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase,Acorn_,Acorn_grouped_,time_,stdorToU_,icon_,LCLid_
852288,14.11,237,4.05,0.35,8.12,1013.10,12.81,0.64,4,8.48,0.25,4,1,469,1,5,163
139358,12.32,300,4.10,0.20,2.36,1002.50,11.86,0.80,2,2.81,0.96,4,1,285,1,3,62
718631,16.06,234,9.18,0.49,7.68,992.50,11.20,0.81,1,9.00,0.80,3,1,662,1,5,745
136727,18.02,253,12.51,0.72,1.24,1019.29,9.24,0.87,2,12.58,0.24,4,1,616,1,3,306
119517,11.47,281,8.92,0.58,3.78,1010.80,9.38,0.92,1,8.87,0.31,4,1,647,1,3,262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137732,8.99,89,0.86,0.27,3.66,1013.74,7.02,0.78,2,2.14,0.74,4,1,426,1,3,247
125427,5.84,320,0.73,0.36,3.20,1012.58,12.92,0.83,1,1.63,0.53,4,1,331,1,3,125
140394,22.92,209,12.14,0.56,2.82,1019.93,12.83,0.69,6,14.04,0.23,4,1,175,1,3,301
155351,12.13,130,8.67,0.67,1.48,1000.65,9.37,0.86,4,9.78,0.62,4,1,509,1,3,19


In [22]:
Y_FS

852288    22
139358     7
718631    21
136727     2
119517     8
          ..
137732     5
125427     4
140394     2
155351     2
149454    11
Name: energy_sum, Length: 4429, dtype: int32

In [23]:
# Selecting top features and calculating their score
top_features= SelectKBest(score_func=chi2, k=3)
top_fit= top_features.fit(X_FS,Y_FS)
Scores= pd.DataFrame(top_fit.scores_)
X_FS_Columns= pd.DataFrame(X_FS.columns)

In [24]:
# Sorting these scores with their column names
features_scores= pd.concat([X_FS_Columns, Scores], axis=1)
features_scores.columns= ['Features_name', 'Score']
features_scores.sort_values(by = 'Score')

,Features_name,Score
14,stdorToU_,2.532927e-30
12,Acorn_grouped_,2.532927e-30
7,humidity,1.526567e+00
3,cloudCover,4.123738e+00
15,icon_,8.057842e+00
5,pressure,9.307804e+00
10,moonPhase,1.040090e+01
4,windSpeed,3.690692e+01
11,Acorn_,3.824100e+01
6,visibility,5.587849e+01


In [25]:
#Getting X and Y with selected features.
X2 = X_FS.iloc[:,[16,13,1,0,9]]  
Y2 = Y_FS

In [26]:
X2

,LCLid_,time_,windBearing,temperatureMax,temperatureMin
852288,163,469,237,14.11,8.48
139358,62,285,300,12.32,2.81
718631,745,662,234,16.06,9.00
136727,306,616,253,18.02,12.58
119517,262,647,281,11.47,8.87
...,...,...,...,...,...
137732,247,426,89,8.99,2.14
125427,125,331,320,5.84,1.63
140394,301,175,209,22.92,14.04
155351,19,509,130,12.13,9.78


In [27]:
Y2

852288    22
139358     7
718631    21
136727     2
119517     8
          ..
137732     5
125427     4
140394     2
155351     2
149454    11
Name: energy_sum, Length: 4429, dtype: int32

In [28]:
#Splitting dataset in training and testing
X_train,X_test,y_train,y_test=train_test_split(X2,Y2,test_size=0.2,random_state=2005)

In [29]:
#Calculating Mean square error using linear Regression with feature selection.
lin2 = LinearRegression()
lin2.fit(X_train,y_train)
lin2_y_pred = lin2.predict(X_test)
L2 = mean_squared_error(y_test,lin2_y_pred)
L2

53.47840615804803

In [30]:
#Calculating Mean square error using KNN Regression with feature selection.
knn2 = KNeighborsRegressor(n_neighbors=20)
knn2.fit(X_train,y_train)
knn2_y_pred = knn2.predict(X_test)
K2 = mean_squared_error(y_test,knn2_y_pred)
K2

55.472629796839726

In [31]:
#Calculating Mean square error using Ridge Regression with feature selection.
ridg2 = Ridge(alpha=1.0)
ridg2.fit(X_train,y_train)
ridg2_y_pred = ridg2.predict(X_test)
R2 = mean_squared_error(y_test,ridg2_y_pred)
R2

53.478404665990396

In [32]:
# Best Model among the models with and without feature selection.
Models = {"Linear Regression without feature selection ":L1,"Linear Regression with feature selection ":L2,
          "KNN Regression without feature selection ":K1,"KNN Regression with feature selection ":K2,
          "Ridge Regression without feature selection ":R1,"Ridge Regression with feature selection ":R2}
Best_Model_value = min(Models.values())
for i in Models :
    if Models[i]==min(Models.values()):
        print(i + 'is the Best Model for the dataset used.')

Ridge Regression with feature selection is the Best Model for the dataset used.


## Reference :


https://365datascience.com/tutorials/python-tutorials/predictive-model-python/

https://scikit-learn.org/stable/modules/feature_selection.html

https://github.com/Gaelim/PowerBI_Python_A_to_Z